**Implemented By Muhammad Hanan Asghar**

In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 75 bytes


In [2]:

import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"# /content/gdrive/My Drive/Kaggle is the path where kaggle.json is present in the Google Drive

!kaggle datasets download -d nabeelsajid917/covid-19-x-ray-10000-images

 88% 82.0M/92.7M [00:00<00:00, 86.9MB/s]
100% 92.7M/92.7M [00:00<00:00, 189MB/s] 


In [ ]:
#unzipping the zip files and deleting the zip files
!unzip \*.zip  && rm *.zip

In [11]:
!pip install -q keras-tuner

     |████████████████████████████████| 71kB 7.5MB/s 


In [12]:
import keras
import os
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from kerastuner import RandomSearch

In [13]:
import cv2
img = cv2.imread("/content/dataset/covid/covid (1).jpeg")
print(img.shape)

(842, 1024, 3)


In [21]:
def Model(hp):
  cnn = Sequential()
  cnn.add(Conv2D(filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
                 kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
                 input_shape=(150, 150, 3), 
                 activation='relu')) 
  
  cnn.add(MaxPooling2D(pool_size=(2, 2)))
   
  cnn.add(Conv2D(filters=hp.Int('conv_2_filter', min_value=32, max_value=128, step=16),
                 kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
                 activation='relu'))
  
  cnn.add(MaxPooling2D(pool_size=(2, 2)))
  
  cnn.add(Flatten())

  cnn.add(Dense(units=hp.Int('dense_1_units', 
                min_value=512, max_value=1024, step=16),
                activation='relu'))
  cnn.add(Dropout(rate=0.5))

  cnn.add(Dense(units=2, activation='softmax'))

  cnn.compile(optimizer=keras.optimizers.RMSprop(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  return cnn

In [26]:
!rm -rf /content/outputs

In [27]:
tuner = RandomSearch(Model,
                    objective='accuracy',
                    max_trials=5,
                    directory='/content/outputs',
                    project_name="Corona Detection")

In [28]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode="nearest")
  
training_set = train_datagen.flow_from_directory(
        "/content/dataset",
        target_size=(150, 150),
        batch_size=10,
        class_mode='categorical')

Found 98 images belonging to 2 classes.


In [29]:
tuner.search(training_set,
            epochs=4)

Trial 5 Complete [00h 01m 40s]
accuracy: 0.7551020383834839

Best accuracy So Far: 0.8163265585899353
Total elapsed time: 00h 03m 12s
INFO:tensorflow:Oracle triggered exit


In [30]:
model = tuner.get_best_models(num_models=1)[0]

In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 112)     3136      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 112)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        32288     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 41472)             0         
_________________________________________________________________
dense (Dense)                (None, 656)               27206288  
_________________________________________________________________
dropout (Dropout)            (None, 656)               0

In [32]:
model.fit(training_set,
          epochs=10,
          initial_epoch=4)

Epoch 5/10
10/10 [==============================] - 4s 335ms/step - loss: 0.7046 - accuracy: 0.7285
Epoch 6/10
10/10 [==============================] - 3s 302ms/step - loss: 0.6250 - accuracy: 0.7848
Epoch 7/10
10/10 [==============================] - 3s 295ms/step - loss: 1.2896 - accuracy: 0.7361
Epoch 8/10
10/10 [==============================] - 3s 311ms/step - loss: 0.5236 - accuracy: 0.8841
Epoch 9/10
10/10 [==============================] - 3s 309ms/step - loss: 0.6480 - accuracy: 0.8191
Epoch 10/10
10/10 [==============================] - 3s 302ms/step - loss: 0.5486 - accuracy: 0.7611


In [33]:
training_set.class_indices

{'covid': 0, 'normal': 1}

In [42]:
# Prediction for Image using our trained model
import numpy as np
from keras.preprocessing import image
test_image = image.load_img("/content/dataset/covid/covid (14).jpeg", target_size=(150,150))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)

In [43]:
if int(round(result[0][0])) == 1:
  print("The Patient Has Corona.")
elif int(round(result[0][1])) == 1:
  print("The Patient Has Not Corona")

The Patient Has Corona.


In [44]:
# Prediction for Image using our trained model
import numpy as np
from keras.preprocessing import image
test_image = image.load_img("/content/dataset/normal/IM-0227-0001.jpeg", target_size=(150,150))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)

In [45]:
if int(round(result[0][0])) == 1:
  print("The Patient Has Corona.")
elif int(round(result[0][1])) == 1:
  print("The Patient Has Not Corona")

The Patient Has Not Corona


In [53]:
# Prediction for Image using our trained model
import numpy as np
from keras.preprocessing import image
test_image = image.load_img("/content/41.jpeg", target_size=(150,150))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)

In [54]:
if int(round(result[0][0])) == 1:
  print("The Patient Has Corona.")
elif int(round(result[0][1])) == 1:
  print("The Patient Has Not Corona")

The Patient Has Not Corona


In [52]:
result

array([[1., 0.]], dtype=float32)